## Boilerplate

In [1]:
!git clone https://github.com/nateraw/huggingface-vit-finetune.git

fatal: destination path 'huggingface-vit-finetune' already exists and is not an empty directory.


In [2]:
%cd huggingface-vit-finetune/
!pip install -r requirements.txt
!pip install git+https://github.com/huggingface/transformers.git@master --upgrade

/content/huggingface-vit-finetune
  Cloning https://github.com/huggingface/transformers.git (to revision master) to /tmp/pip-req-build-chc8zxpy
  Running command git clone -q https://github.com/huggingface/transformers.git /tmp/pip-req-build-chc8zxpy
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for transformers: filename=transformers-4.6.0.dev0-cp37-none-any.whl size=2106690 sha256=ea94ea15e3f9aca002bc89918bd4fb1fde6c6daaefed5371952374201c4a354d
  Stored in directory: /tmp/pip-ephem-wheel-cache-rwv_sbul/wheels/03/01/00/f2c9020459e177a2729b25f0b9628d95bb967727d71a118170
Successfully built transformers
  Found existing installation: transformers 4.6.0.dev0
    Uninstalling transformers-4.6.0.dev0:
      Successfully uninstalled transformers-4.6.0.dev0


In [3]:
from transformers import ViTFeatureExtractor, ViTForImageClassification
from PIL import Image
import requests

In [4]:
!pip install adversarial-robustness-toolbox
!pip install timm
!pip install --upgrade matplotlib

Requirement already up-to-date: matplotlib in /usr/local/lib/python3.7/dist-packages (3.4.1)


In [5]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import torchvision
import torchvision.transforms as transforms
import torchvision.datasets as datasets

import matplotlib
import matplotlib.pyplot as plt
import numpy as np

import art
from art.attacks.evasion import FastGradientMethod, ProjectedGradientDescentPyTorch
from art.estimators.classification import PyTorchClassifier

import urllib
from PIL import Image

# For 98% model
from transformers import ViTFeatureExtractor, ViTForImageClassification
import requests

from torch.utils.data import DataLoader

In [6]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

## Load in the Pretrained VIT (Cifar 10) Network

In [7]:
feature_extractor = ViTFeatureExtractor.from_pretrained('nateraw/vit-base-patch16-224-cifar10')
model = ViTForImageClassification.from_pretrained('nateraw/vit-base-patch16-224-cifar10')

In [8]:
feature_extractor  # Check the Feature Extractor Params

ViTFeatureExtractor {
  "do_normalize": true,
  "do_resize": true,
  "feature_extractor_type": "ViTFeatureExtractor",
  "image_mean": [
    0.5,
    0.5,
    0.5
  ],
  "image_std": [
    0.5,
    0.5,
    0.5
  ],
  "resample": 2,
  "size": 224
}

In [9]:
model  # look at the model

ViTForImageClassification(
  (vit): ViTModel(
    (embeddings): ViTEmbeddings(
      (patch_embeddings): PatchEmbeddings(
        (projection): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
      )
      (dropout): Dropout(p=0.0, inplace=False)
    )
    (encoder): ViTEncoder(
      (layer): ModuleList(
        (0): ViTLayer(
          (attention): ViTAttention(
            (attention): ViTSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
            )
            (output): ViTSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
            )
          )
          (intermediate): ViTIntermediate(
            (dense): Linear(in_features=768, out_fea

In [10]:
model.config.id2label = {
    "0": "airplane",
    "1": "automobile",
    "2": "bird",
    "3": "cat",
    "4": "deer",
    "5": "dog",
    "6": "frog",
    "7": "horse",
    "8": "ship",
    "9": "truck"
  }

model.config.label2id = {
    "airplane": 0,
    "automobile": 1,
    "bird": 2,
    "cat": 3,
    "deer": 4,
    "dog": 5,
    "frog": 6,
    "horse": 7,
    "ship": 8,
    "truck": 9
  }

model.config

ViTConfig {
  "_name_or_path": "nateraw/vit-base-patch16-224-cifar10",
  "architectures": [
    "ViTForImageClassification"
  ],
  "attention_probs_dropout_prob": 0.0,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.0,
  "hidden_size": 768,
  "id2label": {
    "0": "airplane",
    "1": "automobile",
    "2": "bird",
    "3": "cat",
    "4": "deer",
    "5": "dog",
    "6": "frog",
    "7": "horse",
    "8": "ship",
    "9": "truck"
  },
  "image_size": 224,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "airplane": 0,
    "automobile": 1,
    "bird": 2,
    "cat": 3,
    "deer": 4,
    "dog": 5,
    "frog": 6,
    "horse": 7,
    "ship": 8,
    "truck": 9
  },
  "layer_norm_eps": 1e-12,
  "model_type": "vit",
  "num_attention_heads": 12,
  "num_channels": 3,
  "num_hidden_layers": 12,
  "patch_size": 16,
  "transformers_version": "4.6.0.dev0"
}

Load Dataset


In [11]:
# Note: feature_extractor extractor does the tranforms per batch, so no need to
# do transforms on the dataset

!pip install datasets

     |████████████████████████████████| 194kB 19.9MB/s 
     |████████████████████████████████| 245kB 41.2MB/s 


In [12]:
from datasets import load_dataset

# train = load_dataset('cifar10', split='train')
# test = load_dataset('cifar10', split='test')

# train.set_format(type='torch', columns=None)  # None == return all cols
# test.set_format(type='torch', columns=None)  # None == return all cols

In [13]:
cifar_trainset = datasets.CIFAR10(root='./data', train=True, download=True, transform=feature_extractor)
cifar_testset = datasets.CIFAR10(root='./data', train=False, download=True, transform=feature_extractor)

cifar_trainloader = torch.utils.data.DataLoader(cifar_trainset, batch_size=1,
                                          shuffle=True, num_workers=0)
cifar_testloader = torch.utils.data.DataLoader(cifar_testset, batch_size=1,
                                         shuffle=False, num_workers=0)


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


In [14]:
# print("\n👉Slice of the two items 'dataset[10:12]':")
# pprint(train[10:12])

In [15]:
# cifar_trainloader = torch.utils.data.DataLoader(train, batch_size=16,
#                                           shuffle=True, num_workers=0)
# cifar_testloader = torch.utils.data.DataLoader(test, batch_size=16,
#                                          shuffle=False, num_workers=0)

Set Everything that needs to be set to(deevice)

In [16]:
model.to(device)

ViTForImageClassification(
  (vit): ViTModel(
    (embeddings): ViTEmbeddings(
      (patch_embeddings): PatchEmbeddings(
        (projection): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
      )
      (dropout): Dropout(p=0.0, inplace=False)
    )
    (encoder): ViTEncoder(
      (layer): ModuleList(
        (0): ViTLayer(
          (attention): ViTAttention(
            (attention): ViTSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
            )
            (output): ViTSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
            )
          )
          (intermediate): ViTIntermediate(
            (dense): Linear(in_features=768, out_fea

## Setup Adverarial Attack

In [17]:
result_dict = {}

In [18]:
class Transformer(nn.Module):
  def __init__(self, model):
    super(Transformer, self).__init__()
    self.model = model
  def forward(self, x):
    return model(x)[0]
transformer = Transformer(model)

In [19]:
transformer.cpu()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

transformer_classifier = PyTorchClassifier(
    model=transformer,
    loss=criterion,
    optimizer=optimizer,
    input_shape=(3, 224, 224),
    nb_classes=10,
)

In [ ]:
# cifar-10 DATASET consists of 60000 images
correct = 0
total = 0

with torch.no_grad():
    i = 0;
    for inputs, labels in cifar_testloader:
        inputs, labels = inputs['pixel_values'][0].to(device), labels.to(device)

        outputs = transformer(inputs)

        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

        i = i + 1;
        if(i % 100 == 0):
            print('Accuracy of the network on %d cifar-10 test images: %f %%' % (total, 100 * correct / total))


print('Accuracy of the network on %d cifar-10 test images: %f %%' % (
    total,
    100 * correct / total))

result_dict["Transformer"] = 100 * correct / total

In [20]:
from art.attacks.evasion import ZooAttack

attack_ZOO = ZooAttack(
        classifier=transformer_classifier,
        max_iter=20,
        batch_size=1,
        learning_rate=1e-2,
        targeted=False,
        abort_early=True
)

In [ ]:
# Evaluation

correct = 0
total = 0

with torch.no_grad():
    i = 0;
    for inputs, labels in cifar_testloader:
        # inputs, labels = inputs['pixel_values'][0].to(device), labels.to(device)
        inputs, labels = inputs['pixel_values'][0].cpu(), labels.numpy()
 
        adversarial_inputs = attack_ZOO.generate(x=inputs)
        outputs = transformer_classifier.predict(adversarial_inputs)

        predicted = np.argmax(outputs, 1)
        total += labels.size
        correct += (predicted == labels).sum().item()

        print('Accuracy of the network on %d cifar-10 test images: %f %%' % (total, 100 * correct / total))


Accuracy of the network on 1 cifar-10 test images: 100.000000 %



Accuracy of the network on 2 cifar-10 test images: 100.000000 %



Accuracy of the network on 3 cifar-10 test images: 100.000000 %



Accuracy of the network on 4 cifar-10 test images: 100.000000 %



Accuracy of the network on 5 cifar-10 test images: 100.000000 %


In [1]:
print('Accuracy of the network on %d cifar-10 adversaril images: %f %%' % (
    total,
    100 * correct / total))

NameError: ignored

In [ ]:
result_dict["ZOO_10_iters"] = 100 * correct / total

In [22]:
labels

array([3])